In [1]:
import torch
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch.nn.functional as F

from torch import nn
from torchvision import datasets, transforms
from PIL import Image

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping

In [2]:
# bez tego wysadza kernel kiedy rysuje obrazek
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [27]:
classes = {
    0: "plane",
    1: "car",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
batch_size = 64
torch.manual_seed(0)

cuda:0


In [4]:
transform = transforms.Compose([
    transforms.ToTensor()  # this includes scaling to [0, 1]
])

train_data = datasets.CIFAR10("./cifar10", download=True, transform=transform, train=True)
test_data = datasets.CIFAR10("./cifar10", download=True, transform=transform, train=False)

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(train_data)
print(test_data)
print(train_data[0][0].size())
print(train_data[0][0])

In [7]:
def print_img(img, axs):
    img_np = img.numpy()
    img_denormalized = (img_np*255).astype("uint8").transpose(1, 2, 0) 
    return axs.imshow(img_denormalized)

In [ ]:
fig, axs = plt.subplots(3, 5, constrained_layout=True)
axs = np.reshape(axs, -1)
for x in range(15):
    img, label = train_data[x]
    axs[x].title.set_text(classes[label])
    print_img(img, axs[x])

In [12]:
def run_model(model):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    trainer = create_supervised_trainer(model, optimizer, loss_fn, device)

    val_metrics = {
        "accuracy": Accuracy(),
        "loss": Loss(loss_fn)
    }

    train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
    val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        train_evaluator.run(train_loader)
        metrics = train_evaluator.state.metrics
        print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}")


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        val_evaluator.run(test_loader)
        metrics = val_evaluator.state.metrics
        print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy'] * 100:.2f}%, Avg loss: {metrics['loss']:.2f}")

    def score_function(engine):
        metrics = engine.state.metrics
        return metrics["accuracy"]
    
    val_evaluator.add_event_handler(Events.COMPLETED, EarlyStopping(3, score_function, trainer))

    trainer.run(train_loader, max_epochs=100)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
run_model(model)

In [28]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.conv3 = nn.Conv2d(16, 32, 5)
        # self.conv4 = nn.Conv2d(32, 48, 5)
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 120)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc1 = nn.Linear(6 * 28 * 28, 120)
        # self.fc1 = nn.Linear(32 * 16 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 10)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.conv3(x)
        # x = self.pool(F.relu(self.conv3(x)))
        # x = self.pool(F.relu(self.conv4(x)))
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.relu(self.conv3(x))
        # x = F.relu(self.conv4(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
model = Net().to(device)
run_model(model)

Training Results - Epoch[1] Avg accuracy: 34.45%, Avg loss: 1.67
Validation Results - Epoch[1] Avg accuracy: 34.49%, Avg loss: 1.67
Training Results - Epoch[2] Avg accuracy: 42.32%, Avg loss: 1.51
Validation Results - Epoch[2] Avg accuracy: 42.17%, Avg loss: 1.52
Training Results - Epoch[3] Avg accuracy: 48.83%, Avg loss: 1.40
Validation Results - Epoch[3] Avg accuracy: 48.14%, Avg loss: 1.42
Training Results - Epoch[4] Avg accuracy: 52.23%, Avg loss: 1.31
Validation Results - Epoch[4] Avg accuracy: 51.28%, Avg loss: 1.33
Training Results - Epoch[5] Avg accuracy: 57.18%, Avg loss: 1.19
Validation Results - Epoch[5] Avg accuracy: 55.54%, Avg loss: 1.25
Training Results - Epoch[6] Avg accuracy: 60.39%, Avg loss: 1.11
Validation Results - Epoch[6] Avg accuracy: 58.15%, Avg loss: 1.18
Training Results - Epoch[7] Avg accuracy: 60.29%, Avg loss: 1.10
Validation Results - Epoch[7] Avg accuracy: 57.39%, Avg loss: 1.19
Training Results - Epoch[8] Avg accuracy: 64.53%, Avg loss: 1.00
Validation 

2023-03-26 15:37:59,044 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Validation Results - Epoch[22] Avg accuracy: 64.24%, Avg loss: 1.11


1 + 3 - Validation Results - Epoch[11] Avg accuracy: 55.59%, Avg loss: 1.34
2 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.85%, Avg loss: 1.34

3 + 3 - Validation Results - Epoch[10] Avg accuracy: 58.77%, Avg loss: 1.23
3 + 4 - Validation Results - Epoch[12] Avg accuracy: 61.44%, Avg loss: 1.42
3 + 5 - Validation Results - Epoch[15] Avg accuracy: 57.74%, Avg loss: 1.47

4 + 3 - Validation Results - Epoch[11] Avg accuracy: 59.61%, Avg loss: 1.36
4 + 4 - Validation Results - Epoch[15] Avg accuracy: 61.82%, Avg loss: 1.52
4 + 5 - Validation Results - Epoch[19] Avg accuracy: 60.91%, Avg loss: 1.24

5 + 3 - Validation Results - Epoch[18] Avg accuracy: 60.97%, Avg loss: 1.17
5 + 4 - Validation Results - Epoch[14] Avg accuracy: 60.22%, Avg loss: 1.24
5 + 5 - Validation Results - Epoch[23] Avg accuracy: 61.34%, Avg loss: 1.33
5 + 6 - Validation Results - Epoch[20] Avg accuracy: 59.16%, Avg loss: 1.23

6 + 3 - Validation Results - Epoch[17] Avg accuracy: 60.48%, Avg loss: 1.22


# with pooling
2 + 2 - Validation Results - Epoch[16] Avg accuracy: 61.48%, Avg loss: 1.09
2 + 3 - Validation Results - Epoch[19] Avg accuracy: 61.46%, Avg loss: 1.11
2 + 4 - Validation Results - Epoch[17] Avg accuracy: 61.60%, Avg loss: 1.10
2 + 5 - Validation Results - Epoch[17] Avg accuracy: 58.76%, Avg loss: 1.23

2 + 2 - Validation Results - Epoch[25] Avg accuracy: 63.66%, Avg loss: 1.10  # 3x3 kernel

2 + 2 - Validation Results - Epoch[65] Avg accuracy: 59.91%, Avg loss: 1.17  # sigmoid
2 + 3 - Validation Results - Epoch[31] Avg accuracy: 56.99%, Avg loss: 1.21  # sigmoid
2 + 4 - Validation Results - Epoch[40] Avg accuracy: 59.25%, Avg loss: 1.17  # sigmoid
2 + 2 - Validation Results - Epoch[22] Avg accuracy: 52.06%, Avg loss: 1.33  # 3x3 kernel sigmoid

3 + 4 - Validation Results - Epoch[15] Avg accuracy: 65.09%, Avg loss: 1.09  # 3x3 kernel, no pooling on last conv, no relu between conv and pool
3 + 5 - Validation Results - Epoch[20] Avg accuracy: 65.20%, Avg loss: 1.11  # 3x3 kernel, no pooling on last conv, no relu between conv and pool
3 + 5 - Validation Results - Epoch[22] Avg accuracy: 64.24%, Avg loss: 1.11  # 3v3 kernel, no pooling on last conv